In [ ]:
import torch
import numpy as np
from PIL import Image

from input_pipeline import Images
from generator import Generator
from train import Simplifier

In [ ]:
DEVICE = torch.device('cpu')
simplifier = Simplifier()

# Load images

In [ ]:
WIDTH, HEIGHT = 1024, 1024
DATA = '/home/dan/datasets/patterns/images/'
dataset = Images(folder=DATA, size=(HEIGHT, WIDTH))

B = dataset[0].unsqueeze(0)

# s = 1.0
# image = Image.open('/home/dan/datasets/COCO/images/train2017/000000458625.jpg')
# w, h = image.size
# WIDTH, HEIGHT = int(s * w), int(s * h)
# image = image.resize((WIDTH, HEIGHT))
# image = np.array(image, dtype='float32')/255.0
# B = torch.FloatTensor(image).permute(2, 0, 1).unsqueeze(0)

A = simplifier(B).to(DEVICE)
B = B.to(DEVICE)

# Show images

In [ ]:
edges = A.squeeze(0).cpu().permute(1, 2, 0).numpy()[:, :, 0]
image = B.squeeze(0).cpu().permute(1, 2, 0).numpy()

edges = (edges * 255).astype('uint8')
image = (image * 255).astype('uint8')

Image.fromarray(image)

In [ ]:
Image.fromarray(edges)

# Load the model

In [ ]:
G = Generator(1, 3).eval().to(DEVICE)
G.load_state_dict(torch.load('models/run00_epoch_100_generator.pth', map_location=DEVICE))

# Generate

In [ ]:
with torch.no_grad():
    B_generated = G(A)

image = B_generated.squeeze(0).permute(1, 2, 0).cpu().numpy()
image = (image * 255).astype('uint8')
Image.fromarray(image)

# Generate iteratively

In [ ]:
num_iterations = 3
grid = Image.new('RGB', (WIDTH, num_iterations * HEIGHT))

A_current = A.clone()
for i in range(num_iterations):

    with torch.no_grad():
        B_generated = G(A_current)
        B_generated = B_generated.cpu()
        A_current = simplifier(B_generated).to(DEVICE)

    image = B_generated.squeeze(0).permute(1, 2, 0).numpy()
    image = (image * 255).astype('uint8')
    image = Image.fromarray(image)
    grid.paste(image, (0, i * HEIGHT))     

grid